In [24]:
#consumer
lista_msg = []

from kafka import KafkaConsumer
import json
from pprint import pprint

consumer = KafkaConsumer(
  bootstrap_servers=["kafka:9092"],
  auto_offset_reset="latest",
  value_deserializer=lambda v: json.loads(v.decode("utf-8")))

consumer.subscribe(["test_json"])

for msg in consumer:
  lista_msg.append(msg)


KeyboardInterrupt: 

In [5]:
#importare le librerie per caricare i file dal consumer a mongo
from pymongo import MongoClient

In [6]:
client = MongoClient('mongo', 27017, username = 'admin', password = 'DataMan2019!')

In [7]:
twitter_indicator = client.twitter_indicator
indicator = twitter_indicator.indicator

In [8]:
indicator

Collection(Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'twitter_indicator'), 'indicator')

In [9]:
import pandas as pd

In [26]:
data = pd.DataFrame(lista_msg)

In [13]:
lista_citta = {"Amsterdam" : "52.22 , 4.53", "Athens" : "37.58 , 23.42" , "Barcelona" :"41.22 , 2.10" , "Berlin" : "52.30  , 13.25" ,
       "Bratislava" : "48.09 , 17.07",  "Brussels"  : "50.50 , 4.20" , "Budapest" : "47.3 , 19.05" , "Copenhagen" : "55.4 , 12.35"  ,
       "Dublin"  : "53.20 , 6.15"  , "Edinburgh" : "55.57 , -3.11"   ,  "Geneva"   : "46.12 , 6.09"  ,  "Hamburg" : "53.34 , 10.0" , "Helsinki" : "60.10 , 24.58" ,             
       "Krakow" : "50.3 , 19.56" ,  "Lisbon" : "38.43 , 9.08" ,  "Ljubljana" : "46.03 , 14.31" ,  "London" : "51.3 , 0.10" , "Luxembourg" : "49.45 , 6.05",
       "Lyon" : "45.46 , 4.50" , "Madrid" : "40.24, 3.41"   ,    "Milan" : "45.28, 9.10"  ,  "Munich" : "48.08 , 11.34 " ,   
        "Oporto"  : "41.09 , 8.38"  , "Oslo" : "59.55 , 10.45" ,  "Paris" : "48.52 , 2.20" ,
       "Prague"  : "50.05 , 14.26" ,     "Rome" : "41.54 , 12.29","Stockholm" : " 59.20 , 18.03","Vienna" : "48.13 , 16.20","Warsaw" : "52.13 , 21.00","Zurich" : "47.22 , 8.32"}

In [28]:
#trasformazione di tutte le città in maiuscolo per futura integration
lista_maiusc = []
for el in lista_citta.keys():
    lista_maiusc.append(el.upper())

In [29]:
lista_maiusc

['AMSTERDAM',
 'ATHENS',
 'BARCELONA',
 'BERLIN',
 'BRATISLAVA',
 'BRUSSELS',
 'BUDAPEST',
 'COPENHAGEN',
 'DUBLIN',
 'EDINBURGH',
 'GENEVA',
 'HAMBURG',
 'HELSINKI',
 'KRAKOW',
 'LISBON',
 'LJUBLJANA',
 'LONDON',
 'LUXEMBOURG',
 'LYON',
 'MADRID',
 'MILAN',
 'MUNICH',
 'OPORTO',
 'OSLO',
 'PARIS',
 'PRAGUE',
 'ROME',
 'STOCKHOLM',
 'VIENNA',
 'WARSAW',
 'ZURICH']

In [30]:
diz = {}

for el in lista_maiusc:
    lista_find = data['value'].str.upper().str.find(el)
    count = 0
    for elemento in lista_find:
        if elemento >= 0:
            count += 1
    diz[el] = count
    

In [32]:
#prova dizionario con il rispettivo numero di tweet per ogni città 
diz

{'AMSTERDAM': 302,
 'ATHENS': 1,
 'BARCELONA': 11857,
 'BERLIN': 2134,
 'BRATISLAVA': 381,
 'BRUSSELS': 20,
 'BUDAPEST': 2675,
 'COPENHAGEN': 16,
 'DUBLIN': 13,
 'EDINBURGH': 11,
 'GENEVA': 795,
 'HAMBURG': 1942,
 'HELSINKI': 34,
 'KRAKOW': 9,
 'LISBON': 4,
 'LJUBLJANA': 276,
 'LONDON': 178,
 'LUXEMBOURG': 500,
 'LYON': 33,
 'MADRID': 416,
 'MILAN': 3618,
 'MUNICH': 1811,
 'OPORTO': 20,
 'OSLO': 14,
 'PARIS': 265,
 'PRAGUE': 2559,
 'ROME': 124,
 'STOCKHOLM': 160,
 'VIENNA': 2565,
 'WARSAW': 802,
 'ZURICH': 80}

In [33]:
indicator.insert_one(diz)

In [34]:
indicator

Collection(Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'twitter_indicator'), 'indicator')

In [35]:
twitter_indicator

Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'twitter_indicator')

In [51]:
mongo_db = list(client.twitter_indicator.indicator.find())

In [52]:
#prova 
data_indicator1 = pd.DataFrame(mongo_db)
data_indicator1

,_id,AMSTERDAM,ATHENS,BARCELONA,BERLIN,BRATISLAVA,BRUSSELS,BUDAPEST,COPENHAGEN,DUBLIN,...,MUNICH,OPORTO,OSLO,PARIS,PRAGUE,ROME,STOCKHOLM,VIENNA,WARSAW,ZURICH
0,5e15a938fd6630dcf43ee451,411,19,3162,2401,427,19,2806,18,11,...,2111,7,13,243,2716,90,2622,2531,936,92
1,5e15adfafd6630dcf43ee452,421,6,12024,2190,415,17,830,16,8,...,1698,9,37,203,2877,105,2978,2455,823,74
2,5e15b807fd6630dcf43ee453,434,8,12723,2536,253,30,2482,12,26,...,1945,12,21,268,2531,128,1160,2174,836,86
3,5e15bf2bfd6630dcf43ee454,533,14,17628,2201,485,32,2545,14,19,...,2095,21,34,286,3520,164,2816,2511,1010,72
4,5e15dfe86bdcdf6bd54447f2,432,10,12784,2206,451,24,2600,24,12,...,1669,12,29,324,2631,153,2481,2188,888,104
5,5e15ea646bdcdf6bd54447f3,302,1,11857,2134,381,20,2675,16,13,...,1811,20,14,265,2559,124,160,2565,802,80
